In [24]:
import pandas as pd
import gensim
from collections import namedtuple
import numpy as np

In [25]:
recipes_df = pd.read_csv("clean_data.csv")

In [26]:
def clean_list(ingredient_list):
    ingredient_list = ingredient_list[2:-2]
    ingredient_list = ''.join(ingredient_list)
    ingredient_list = ingredient_list.split("', '")
    return ingredient_list

In [27]:
recipes_df["Clean_bag"]=recipes_df["Bag_Of_Ingredients"].map(clean_list)

In [180]:
bag_of_ingredients = recipes_df["Clean_bag"][0:2000]
bag_of_ingredients

0       [chicken, acorn, squash, sage, rosemary, butte...
1       [egg, whites, new, potatoes, ground, black, pe...
2       [evaporated, milk, milk, garlic, powder, onion...
3       [round, italian, loaf, olive, oil, sweet, ital...
4       [dark, brown, bourbon, fresh, lemon, juice, ap...
                              ...                        
1995    [olive, oil, white, onion, garlic, cloves, cor...
1996    [dried, chiles, árbol, seeds, ribs, tomatillos...
1997    [soy, sauce, sesame, oil, red, pepper, garlic,...
1998    [pork, shoulder, boston, butt, white, onion, r...
1999    [allpurpose, flour, stick, butter, granulated,...
Name: Clean_bag, Length: 2000, dtype: object

In [181]:
' '.join(bag_of_ingredients[0])

'chicken acorn squash sage rosemary butter ground allspice red pepper ground black pepper loaf sturdy white bread apples gala pink lady extravirgin olive oil red onion apple cider vinegar white miso allpurpose flour butter dry white wine chicken broth white miso ground pepper'

In [152]:
bag_of_ingredients = [' '.join(i) for i in bag_of_ingredients]

In [182]:
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(bag_of_ingredients):
    tags = [i]
    docs.append(analyzedDocument(text, tags))

In [184]:
docs[0]

AnalyzedDocument(words=['chicken', 'acorn', 'squash', 'sage', 'rosemary', 'butter', 'ground', 'allspice', 'red', 'pepper', 'ground', 'black', 'pepper', 'loaf', 'sturdy', 'white', 'bread', 'apples', 'gala', 'pink', 'lady', 'extravirgin', 'olive', 'oil', 'red', 'onion', 'apple', 'cider', 'vinegar', 'white', 'miso', 'allpurpose', 'flour', 'butter', 'dry', 'white', 'wine', 'chicken', 'broth', 'white', 'miso', 'ground', 'pepper'], tags=[0])

In [157]:
model = gensim.models.doc2vec.Doc2Vec(docs,vector_size=50, min_count=5, epochs=50)

In [125]:
model.dv[0]

array([ 0.24197641, -0.02445859, -0.00326098,  0.06139241, -0.07989021,
       -0.22866349, -0.17488764, -0.03433101, -0.1289581 , -0.5137616 ,
       -0.10316258,  0.02797434, -0.24201329,  0.04295552, -0.08299074,
       -0.3438963 , -0.26116714,  0.29388008, -0.07158969, -0.55803204,
       -0.11007298, -0.07274986,  0.4576786 , -0.04881627, -0.33012903,
        0.01321315, -0.5123358 , -0.02412922,  0.10079105,  0.41407886,
        0.37535805, -0.23942783, -0.21409446, -0.16591   ,  0.08921493,
       -0.4045692 , -0.70570284, -0.11214074,  0.08760179,  0.08898778,
        0.11049935,  0.12668572, -0.1438299 ,  0.04167461,  0.3837487 ,
        0.07198523, -0.0196912 ,  0.04038083,  0.4233031 , -0.34273905],
      dtype=float32)

In [126]:
ingredients = ["parmisan","cheese","avocado","bread","mushrooms","beans","sausage"]

In [127]:
av_in_vector = model.infer_vector(ingredients)

In [128]:
from numpy import dot
from numpy.linalg import norm

def similar_recipe(ingredients):
    cos_sim = []
    for i in range(0,len(model.dv)):
            cos_sim.append(np.dot(ingredients,model.dv[i])/(norm(ingredients)*norm(model.dv[i])))
    dis_array = np.array(cos_sim)
    return dis_array

In [129]:
def getListofRecipes(cos_sim,n):
    n_index = (-cos_sim).argsort()[:n]
    titles = []
    for i in n_index:
        titles.append(recipes_df.iloc[i,1:3])
    return titles

In [130]:
pd.DataFrame(getListofRecipes(similar_recipe(av_in_vector),10))

,Title,Ingredients
679,Cucumber-Seaweed Salad,"['3/4 ounce dried wakame seaweed, cut into 1-i..."
1223,Romano Beans with Mustard Vinaigrette and Walnuts,"['1 cup walnuts', '3 lb. Romano beans or green..."
168,Sorrel (Hibiscus) Tea,"['2 cups dried hibiscus (sorrel) flowers', '1⁄..."
656,Grilled Salt-and-Pepper Black Bass with Curry ...,"['1 cup grapeseed or other neutral oil, divide..."
977,Warm-Spiced Saucy Lamb Stew,"['1/2 medium head of cauliflower', '3 Tbsp. ex..."
301,Pickle Brine Spice Rub,"['4 Tbsp. coriander seeds', '6 Tbsp. vinegar p..."
491,Lebkuchen (German Honey Bars),"['1 cup (320g) honey', '2 tablespoons (30g) wa..."
1536,Seared Radicchio and Roasted Beets,"['6 medium beets, scrubbed', '5 tablespoons ex..."
1007,Costa Rican Breakfast Bowl (Gallo Pinto),"['1/2 yellow onion, cut into 1/4-inch dice (ab..."
178,Cornbread Muffins With Whipped Sweet Corn,"['3 tablespoons coconut oil, plus more for gre..."


# Test doc2vec model

In [145]:
import os, sys
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
parent_path = os.path.split(os.getcwd())[0]
if parent_path  not in sys.path:
    sys.path.append(parent_path)

In [10]:
from wots_cookin.data import load_data, load_full_stopwords, remove_formatting, load_clean_data, doc2vec_preprocessing
from wots_cookin.doc2vec_trainer import *

In [11]:
df = load_clean_data()

8 rows containing NAs dropped...
6 rows containing empty ingredients dropped...
Data loaded.
Cleaning formatting...
Returning list of 461 stopwords
Removing stopwords...
Order's up!
Returning dataframe with Bag_Of_Ingredients


In [12]:
df.head()

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","[chicken, acorn, squash, sage, rosemary, butte..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...","[egg, whites, new, potatoes, ground, black, pe..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...","[evaporated, milk, milk, garlic, powder, onion..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...","[round, italian, loaf, olive, oil, sweet, ital..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","[dark, brown, bourbon, fresh, lemon, juice, ap..."


In [189]:
bag_of_ingredients = df['Bag_Of_Ingredients'][0:2000]

In [170]:
model = Doc2VecTrainer()

In [190]:
bag_of_ingredients = doc2vec_preprocessing(bag_of_ingredients)

In [191]:
bag_of_ingredients[0]

AnalyzedDocument(words=['chicken', 'acorn', 'squash', 'sage', 'rosemary', 'butter', 'ground', 'allspice', 'red', 'pepper', 'ground', 'black', 'pepper', 'loaf', 'sturdy', 'white', 'bread', 'apples', 'gala', 'pink', 'lady', 'extravirgin', 'olive', 'oil', 'red', 'onion', 'apple', 'cider', 'vinegar', 'white', 'miso', 'allpurpose', 'flour', 'butter', 'dry', 'white', 'wine', 'chicken', 'broth', 'white', 'miso', 'ground', 'pepper'], tags=[0])

In [192]:
model.train_model(bag_of_ingredients)

Recipes corpus set
Model trained!


In [193]:
model.model.dv

In [194]:
ingredients =[ 'chicken', 'acorn', 'squash', 'sage', 'rosemary', 'butter', 'ground', 'allspice', 'red', 'pepper', 'ground', 'black', 'pepper', 'loaf', 'sturdy', 'white', 'bread', 'apples', 'gala', 'pink', 'lady', 'extravirgin', 'olive', 'oil', 'red', 'onion', 'apple', 'cider', 'vinegar', 'white', 'miso', 'allpurpose', 'flour', 'butter', 'dry', 'white', 'wine', 'chicken', 'broth', 'white', 'miso', 'ground', 'pepper']

In [195]:
model.get_ingredients_vector(ingredients)

array([-0.00475507,  0.11997607, -0.09517194,  0.02346379,  0.03063754,
       -0.01324059,  0.01678705, -0.06888784, -0.06190748,  0.01709206,
        0.05240265,  0.04429502, -0.11902413, -0.0778882 , -0.07986069,
        0.17412157, -0.02143359,  0.05140143, -0.05761551, -0.07579109,
       -0.08365007,  0.03830429, -0.09540305, -0.15589367, -0.086905  ,
       -0.00526428,  0.18678032, -0.11180825,  0.09969003, -0.11832611,
       -0.06010412, -0.04320149,  0.07060476,  0.13128188, -0.02126477,
       -0.05280431,  0.03399032,  0.04824045,  0.05225614, -0.07418419,
       -0.03391881,  0.01117652, -0.00925263,  0.05628886, -0.03894513,
       -0.02443234,  0.05185473,  0.096393  , -0.05823968,  0.12364227],
      dtype=float32)

In [196]:
model.get_similarity_score(ingredients)

array([ 0.6970462 ,  0.14644414,  0.06359537, ...,  0.33516115,
       -0.05154458,  0.16619058], dtype=float32)

In [197]:
model.get_similar_recipes(ingredients,df,nrow=30)

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","[chicken, acorn, squash, sage, rosemary, butte..."
1595,Stock-Braised Turkey Legs,['2 tablespoons Diamond Crystal or 1 tablespoo...,Place salt and brown sugar in a small bowl and...,stock-braised-turkey-legs,['2 tablespoons Diamond Crystal or 1 tablespoo...,"[light, brown, turkey, legs, thigh, drumstick,..."
1234,Ramp Fritters,"['1 cup all-purpose flour', '3/4 cup rice flou...",Remove any slippery outer layers from ramp bul...,ramp-fritters,"['1 cup all-purpose flour', '3/4 cup rice flou...","[allpurpose, flour, rice, flour, baking, powde..."
137,Jerk Potato Salad,"['Zest and juice of 2 limes', '2 habanero chil...","Using a fork, mix lime zest and juice, chiles,...",jerk-potato-salad,"['Zest and juice of 2 limes', '2 habanero chil...","[juice, limes, habanero, chiles, seeds, ginger..."
171,Pulled Pork Sandwiches,"['4 tablespoons sweet paprika', '2 tablespoons...","Add paprika, chili powder, cumin, brown sugar,...",pulled-pork-sandwiches-big-hurt-bbq,"['4 tablespoons sweet paprika', '2 tablespoons...","[sweet, paprika, chili, powder, ground, cumin,..."
872,Pimiento Cheese Crackers,"['12 oz. sharp cheddar, coarsely grated (about...","Pulse cheese, flour, salt, and cayenne in a fo...",pimiento-cheese-crackers,"['12 oz. sharp cheddar, coarsely grated (about...","[cheddar, allpurpose, flour, cayenne, pepper, ..."
1586,Make-Ahead Gravy,"['1 tablespoon vegetable oil', '1 pound chicke...",Heat oil in a large saucepan over medium-high....,make-ahead-gravy,"['1 tablespoon vegetable oil', '1 pound chicke...","[vegetable, oil, chicken, wings, onion, carrot..."
1603,Green Bean and Mushroom Casserole with Crispy ...,"['2 pounds mixed wild mushrooms, trimmed, halv...",Preheat oven to 425ºF. Line 2 rimmed baking sh...,green-bean-and-mushroom-casserole-with-crispy-...,"['2 pounds mixed wild mushrooms, trimmed, halv...","[wild, mushrooms, thyme, extravirgin, olive, o..."
1710,Grilled Turkey Under a Brick,['Diamond Crystal or Morton kosher salt (see b...,Determine how much salt you need for your bird...,grilled-turkey-under-a-brick,['Diamond Crystal or Morton kosher salt (see b...,"[ground, mustard, powder, paprika, dried, thym..."
1826,Cinnamon Apple Cupcakes,"['1 cup granulated sugar', '3/4 cup butter, so...",Preheat the oven to 350°F. Line two standard c...,cinnamon-apple-cupcakes,"['1 cup granulated sugar', '3/4 cup butter, so...","[granulated, butter, eggs, vanilla, extract, m..."
